# <center> Data-Driven Reliability Metrics - Analysis </center>
## <center> Week 3 - Part 2</center>
## <center> Interactive Notebook </center>


<center>📚 Source: W3-2 Data-Driven Reliability Metrics - Analysis</center>

## Notebook Overview
In this brief notebook, we'll use what we have learnt in part 1 of week 3 (*W3-1 Data-Driven Reliability Metrics*) to gain insight into our data using visualisation tools. Instead of developing our data preparation and cleaning processes again, we'll load in scripts that contain what we've previously developed (located in the `./scripts` directory).

## Table of Contents
* [3-2.1 - Analysing the effect of our decisions on reliability measures](#3-2.1)
* [3-2.2 - Additional Analysis](#3-2.2)
* [Wrap up and homework](#wrap-up)

## Notebook setup

In [ ]:
!pip install pandas numpy reliability plotly panel nb_black bokeh

In [ ]:
from pathlib import Path

In [ ]:
import pandas as pd
import numpy as np
from reliability.Fitters import Fit_Weibull_2P
import plotly.express as px

import panel as pn
import panel.widgets as pnw

from bokeh.layouts import gridplot, column
from bokeh.io import output_notebook, show
from bokeh.models import ColumnDataSource, DataTable, TableColumn, Column, Row, Slider

In [ ]:
from scripts import data_prep

In [ ]:
pn.extension("plotly")

In [ ]:
output_notebook()

In [ ]:
# Package for ensuring code we write is formatted nicely
%load_ext nb_black

## Load cleaned data

In [ ]:
path_to_data = "../data/rh_mod_v1.csv"
perform_text_cleaning = True

In [ ]:
df = data_prep.prepare_data(filepath=path_to_data, clean_text=perform_text_cleaning)

In [ ]:
df.head()

## 3-2.1 - Analysing the effect of our decisions on reliability measures  <a class="anchor" id="3-2.1"></a>

In this section, we will investigate how the thresholds we set impact the semi-automated reliability meaures we extract from our maintenance work order data.

To make this process interactive, what we developed in the previous notebook (*W3-1 Data-Driven Reliability Metrics*) has been wrapped into the function `get_measures`. This function allows us to pass in information and observe the output of fitting our data to the 2-parameter Weibull distribution. The controllable parameters of this function include:
- end-of-life terms (`eol_terms`), 
- minimum amount of failure/suspension evidence (`min_evidence_points`),
- minimum total actual cost threshold (`cost_threshold`), and
- any given functional location (optional; `functional_loc`).

In [ ]:
def get_measures(
    df: pd.DataFrame,
    eol_terms: list,
    fs_col_name: str = "wo_order_type",
    fs_col_fail_values: list = ["PM01"],
    fs_col_suspension_values: list = ["PM02"],
    min_evidence_points: int = 5,
    cost_threshold: int = 2000,
    functional_loc: str = None,
) -> pd.DataFrame:
    """Function for dynamically calculating reliability measures using expert logic and the Reliability package

    Arguments
        df : a Pandas DataFrame containing maintenance work order data (cleaned or not cleaned)
        eol_terms : a list of terms indicating end-of-life events
        fs_col_name : the name of the column in the supplied DataFrame that can be used to classify end-of-life events as failure or suspension
        fs_col_fail_values : a list of values that will be matched to indicate a failure event
        fs_col_suspension_values : a list of values that will be matched to indicate a suspension event
        min_evidence_points : an integer that indicates the minimum expected failure/suspension evidence required
        cost_threshold : an integer indicating the minimum cost a record with an EOL identified must have to be considered
        functional_loc : a string containing a valid functional location. If this is not supplied, all functional locations will be processed

    Returns
        - df_results_with_objs : Pandas DataFrame with Weibull values for each functional location
        - df_filtered : Pandas DataFrame with original data filtered and classified
    """

    expected_cols = [
        "description",
        "total_actual_costs",
        "functional_loc",
        "actual_start_date",
    ]

    assert set(expected_cols).issubset(
        set(df.columns)
    ), "Data provided to function does not have all the expected columns"

    if functional_loc != None:
        df = df[df["functional_loc"] == functional_loc]

    # Make small dataframe containing object descriptions to join onto results
    df_obj_desc = df[["functional_loc", "object_desc"]].drop_duplicates()

    # Perform preliminary filtering of
    # - functional locations with # records below min_evidence_points

    counts = df["functional_loc"].value_counts()
    idx = counts[counts <= min_evidence_points]
    df_filtered = df[~df["functional_loc"].isin(idx)]

    # Create pattern for matching
    eol_search_pattern = "|".join(eol_terms)

    # Perform EOL identification
    df_filtered["eol"] = df_filtered["description"].str.contains(
        eol_search_pattern, case=False
    )

    # Keep only EOL rows
    df_filtered = df_filtered[df_filtered["eol"]]

    # Perform EOL classification
    df_filtered["fs_clf"] = np.where(
        (df_filtered[fs_col_name].isin(fs_col_suspension_values)),
        "suspension",
        np.where(df_filtered[fs_col_name].isin(fs_col_fail_values), "failure", "other"),
    )

    # Filter data on threshold
    df_filtered = df_filtered[(cost_threshold <= df_filtered["total_actual_costs"])]

    # Groupby functional_loc to get failure/suspension data for reliability measure estimations
    fs_data_per_group = {}
    for name, values in df_filtered.groupby(["functional_loc"]):
        if len(values) < min_evidence_points:
            continue
        else:
            values = values[["actual_start_date", "fs_clf"]]
            # Lets sort the groups data by actual_start_date so that the earliest come first
            fs_data = values.sort_values(by=["actual_start_date"], ascending=True)

            # Calculate the time between event (we'll use days here)
            fs_data["time"] = fs_data["actual_start_date"].diff() / np.timedelta64(
                1, "D"
            )
            fs_data["time"] = fs_data["time"].fillna(0)

            # Encode failures and suspensions as 1 and 0, respectively
            fs_data["fs_clf"].replace(to_replace="failure", value=1, inplace=True)
            fs_data["fs_clf"].replace(to_replace="suspension", value=0, inplace=True)

            fs_data_per_group[name] = fs_data[["fs_clf", "time"]]

    # Get reliability measures from fitting to 2P Weibull distribution
    results_per_group = {}
    for name, fs_data in fs_data_per_group.items():

        num_evidence = len(fs_data)

        if num_evidence < min_evidence_points:
            pass
        else:
            fs_data = fs_data[
                0 < fs_data["time"]
            ]  # Do not take into account any events that have 0 time

            failures = fs_data[fs_data["fs_clf"] == 1]
            right_censored = fs_data[fs_data["fs_clf"] == 0]  # suspensions

            failure_times = failures["time"].tolist()
            right_censored_times = right_censored["time"].tolist()

            try:

                if 1 < len(failures):
                    wbfit = Fit_Weibull_2P(
                        failures=failure_times,
                        right_censored=right_censored_times
                        if len(right_censored_times) > 0
                        else None,
                        show_probability_plot=False,
                        print_results=False,
                    )
                    results_per_group[name] = {
                        "alpha": wbfit.alpha,
                        "beta": wbfit.beta,
                        "mean": wbfit.distribution.mean,
                        "time_on_test": fs_data["time"].sum(),
                        "evidence": num_evidence,
                    }
            except:
                # TODO: handle exceptions with fs data
                pass

    # Join object descriptions onto dataframe for visualisation
    df_results = pd.DataFrame.from_dict(results_per_group).T
    df_results = df_results.rename_axis("functional_loc").reset_index()
    df_results_with_objs = pd.merge(
        df_results, df_obj_desc, on="functional_loc", how="left"
    )

    df_results_with_objs = df_results_with_objs.round(
        {"alpha": 0, "beta": 1, "mean": 0}
    )

    # We will filter out output df_filtered for only the FLOCs that have results
    df_filtered = df_filtered[
        df_filtered["functional_loc"].isin(
            df_results_with_objs["functional_loc"].unique().tolist()
        )
    ]

    return df_results_with_objs, df_filtered

In the function `get_measures`, we can load in a set of end-of-life terms. Recall in the previous notebook, we elicited a set of terms using word embeddings and saved them. If you don't have these, that is okay, but if you do, the following code snippet will load them into this notebook and allow us to use them throughout.

In [ ]:
eol_file_path = "../data/eol_terms.txt"

eol_file = Path(eol_file_path)
if eol_file.is_file():
    print("File found - loading terms")
    with open(eol_file, "r", encoding="utf-8") as infile:
        eol_terms = infile.readlines()
        eol_terms = [
            term.replace("\n", "") for term in eol_terms
        ]  # Remove new line separator

    print(f"Loaded {len(eol_terms)} terms")
else:
    print("No file found - using default terms")
    eol_terms = [
        "replace",
        "change out",
        "c/o",
        "seized",
        "blown",
        "failed",
        "failure",
        "collapsed",
        "holed",
        "unserviceable",
        "u/s",
        "replacement",
    ]

Let's take a look at what our aggregate function is outputting

In [ ]:
# Lets run the function on our data
df_test_results, df_test_clf = get_measures(
    df=df,
    eol_terms=eol_terms,
    fs_col_name="wo_order_type",
    fs_col_fail_values=["PM01"],
    fs_col_suspension_values=["PM02"],
    min_evidence_points=5,
    cost_threshold=10000,
)

In [ ]:
# Output results includes Weibull parameters and other meta data
df_test_results.head(5)

In [ ]:
# Output classifications include classifications for EOL identification and failure suspension
df_test_clf.head(5)

In [ ]:
# You can also get the results for a single floc if you desire
df_test_results_single_floc, df_test_clf_single_floc = get_measures(
    df=df,
    eol_terms=eol_terms,
    fs_col_name="wo_order_type",
    fs_col_fail_values=["PM01"],
    fs_col_suspension_values=["PM02"],
    min_evidence_points=5,
    cost_threshold=10000,
    functional_loc="1071-30-05-01-CVR123-STRC-CHU002",
)

In [ ]:
# Lets look at the 2P Weibull parameter data
df_test_results_single_floc.head()

In [ ]:
# Lets look at the data used to get the 2P Weibull parameters
df_test_clf_single_floc[
    [
        "functional_loc",
        "description",
        "total_actual_costs",
        "actual_start_date",
        "eol",
        "fs_clf",
    ]
].sort_values(by="actual_start_date")

Recall in the previous notebook we set our thresholds to a single values that was applied across all our assets. Obviously, this is not ideal nor representative as the cost of a pump change out is different to that of a gearbox. As indicated in the previous notebook, you could perform an analysis per functional location and update the notebook to account for specific thresholds for each asset type or functional location.

However, what we are going to do here is perform an exploratory visual analysis of our data using an interactive package called [Bokeh Panel](https://panel.holoviz.org/reference/panes/Bokeh.html) and [Plotly](https://plotly.com/python/). If you're familiar with Business Intelligence tools, it will give you the same feel as we'll be creating a small dashboard inside our notebook! This ability to go from raw data all the way to visualisation is seamless in Python, unlike other ardouous workflows.

The code snippet below is a small dashboard/visualisation on top of our `get_measures` function.

In [ ]:
# Set up our interactive application
min_evidence = pnw.IntSlider(name="minimum evidence", value=10, start=1, end=25, step=1)
cost_threshold = pnw.IntSlider(
    name="cost threshold", value=10000, start=0, end=100000, step=2500
)


def mpl_plot(data):
    fig = px.scatter(
        data,
        x="alpha",
        y="beta",
        size="evidence",
        color="mean",
        symbol="object_desc",
        color_continuous_scale="Bluered_r",
        custom_data=["mean", "evidence", "object_desc", "functional_loc"],
    )

    fig.update_traces(
        hovertemplate="<br>".join(
            [
                "Alpha: %{x}",
                "Beta: %{y}",
                "Mean: %{customdata[0]}",
                "Evidence: %{customdata[1]}",
                "Object: %{customdata[2]}",
                "FLOC: %{customdata[3]}",
            ]
        )
    )

    fig = fig.update_layout(title_text=f"Overview of Results ({len(data)} assets)")
    fig = fig.update_layout(coloraxis_colorbar=dict(orientation="h"))
    return fig


def show_measures(min_evidence_points=5, cost_threshold=25000, view_fn=mpl_plot):
    df_test_interactive, df_test_clfs = get_measures(
        df=df,
        eol_terms=eol_terms,
        fs_col_name="wo_order_type",
        fs_col_fail_values=["PM01"],
        fs_col_suspension_values=["PM02"],
        min_evidence_points=min_evidence_points,
        cost_threshold=cost_threshold,
    )

    return view_fn(data=df_test_interactive)


@pn.depends(min_evidence, cost_threshold)
def reactive_measures(min_evidence, cost_threshold):
    return show_measures(
        min_evidence_points=min_evidence, cost_threshold=cost_threshold
    )

Lets run the application and explore our results.

⚠️ Please be aware that every time you interact with the sliders, it will recompute EVERY Weibull estimate for each applicable asset. So please be patient between updates.

In [ ]:
settings = pn.Row(pn.Column(min_evidence, cost_threshold))

app = pn.Column(settings, reactive_measures)
app.servable()

Through the use of interactive visualisation using Bokeh and Plotly we can easily get an intuition for the mean values of a range of assets. For instance, if we set the thresholds very low, we are likely to capture evidence that may not be correct, however as we increase the thresholds our belief in evidence being correct also goes up.

Hence, we can get a feel for the range of mean time estimates on a given asset through this minima/maxima analysis.

### Looking closer at the data

Its apparent that some of the functional locations are displaying interesting behaviour. It would be useful if we could look at each functional location individually to better understand what is happening and improve our understanding. Lets use Bokeh and Plotly to do this.

In [ ]:
min_evidence_2 = pn.widgets.DiscreteSlider(
    name="minimum evidence", options=list(range(0, 25, 1)), value=10
)
cost_threshold_2 = pn.widgets.DiscreteSlider(
    name="cost threshold", value=10000, options=list(range(0, 100000, 2500))
)


def mpl_plot(data):
    fig = px.scatter(
        data,
        x="alpha",
        y="beta",
        size="evidence",
        color="mean",
        symbol="object_desc",
        color_continuous_scale="Bluered_r",
        custom_data=["mean", "evidence", "object_desc", "functional_loc"],
    )

    fig.update_traces(
        hovertemplate="<br>".join(
            [
                "Alpha: %{x}",
                "Beta: %{y}",
                "Mean: %{customdata[0]}",
                "Evidence: %{customdata[1]}",
                "Object: %{customdata[2]}",
                "FLOC: %{customdata[3]}",
            ]
        )
    )

    fig = fig.update_layout(title_text=f"Overview of Results ({len(data)} assets)")
    fig = fig.update_layout(coloraxis_colorbar=dict(orientation="h"))
    return fig


def reactive_dataframe(data):
    # Creating the list of columns:
    columns = [
        TableColumn(field="functional_loc", title="Functional Loc"),
        TableColumn(field="object_desc", title="Object Desc"),
        TableColumn(field="description", title="Description"),
        TableColumn(field="eol", title="Is EOL?"),
        TableColumn(field="total_actual_costs", title="Total Actual Costs"),
        TableColumn(field="fs_clf", title="FS Classification"),
    ]
    # Initializing the table:
    source = ColumnDataSource(data)
    table = DataTable(source=source, columns=columns, width=800, height=200)
    return table


def get_data(min_evidence, cost_threshold):
    df_test_interactive, df_test_clfs = get_measures(
        df=df,
        eol_terms=eol_terms,
        fs_col_name="wo_order_type",
        fs_col_fail_values=["PM01"],
        fs_col_suspension_values=["PM02"],
        min_evidence_points=min_evidence,
        cost_threshold=cost_threshold,
    )
    return df_test_interactive, df_test_clfs


layout = pn.Column(
    pn.Row("### Interactive Visualisation"),
    pn.Row(min_evidence_2, cost_threshold_2),
    pn.Row(
        reactive_dataframe(
            data=get_data(min_evidence_2.value, cost_threshold_2.value)[1]
        )
    ),
    pn.Row(mpl_plot(data=get_data(min_evidence_2.value, cost_threshold_2.value)[0])),
    sizing_mode="stretch_both",
)


def update(event):
    # Update data
    data, data_clf = get_data(min_evidence_2.value, cost_threshold_2.value)
    layout[3][0].object = mpl_plot(data=data)
    layout[2][0].object = reactive_dataframe(data=data_clf)


min_evidence_2.param.watch(update, "value")
cost_threshold_2.param.watch(update, "value")

In [ ]:
# Show the dashboard/application
layout

## 3-2.2 - Additional Analysis<a class="anchor" id="3-2.2"></a>
In this section we'll use our Python programming skills to perform additional analysis on our maintenance work order dataset.

Before we do this, we'll create a dataframe containing the results of our analysis.

In [ ]:
analysis_cost_threshold = 10000
analysis_min_evidence = 10
analysis_eol_terms = eol_terms

In [ ]:
df_analysis_results, df_analysis_filtered = get_measures(
    df=df,
    eol_terms=analysis_eol_terms,
    fs_col_name="wo_order_type",
    fs_col_fail_values=["PM01"],
    fs_col_suspension_values=["PM02"],
    min_evidence_points=analysis_min_evidence,
    cost_threshold=analysis_cost_threshold,
)

In [ ]:
# Lets take a look at the data we are working with
df_analysis_results.head()

In [ ]:
# Sort by mean value
df_analysis_results.sort_values(by=["mean"], inplace=True, ascending=True)

### Find the Top 10 Bad Actors
Here we'll identify the top 10 bad actors in terms of their mean time values (lower value is worse).

In [ ]:
# Top 10 bad actors based on FLOC
top_10_bad_actors_floc = df_analysis_results[:10]
top_10_bad_actors_floc.head(10)

In [ ]:
# Let get the FLOC associated with the worst actor
worst_actor_floc = top_10_bad_actors_floc.iloc[0]["functional_loc"]

In [ ]:
# Lets take a look at the records associated with the functional location
df_analysis_filtered_floc = df_analysis_filtered[
    df_analysis_filtered["functional_loc"] == worst_actor_floc
].sort_values(by=["eol"], ascending=False)
df_analysis_filtered_floc[["description", "total_actual_costs", "eol", "fs_clf"]]

### Failure behavior classification
Here we'll use the alpha and beta values from our analysis to classify all of the applicable assets. Recall that $\beta$ < 1 is early life failures (infant mortality), $\beta$ = 1 is random failures, and $\beta$ > 1 is wear out failures.

In [ ]:
df_analysis_results_fail_clf = df_analysis_results.copy()

In [ ]:
df_analysis_results_fail_clf["behaviour"] = np.where(
    df_analysis_results_fail_clf["beta"] < 1,
    "early_life",
    np.where(df_analysis_results_fail_clf["beta"] == 1, "random", "wear_out"),
)

Lets visualise the failure behaviour / Weibull shape parameter of our assets

In [ ]:
fig = px.bar(
    df_analysis_results_fail_clf,
    x="functional_loc",
    y="mean",
    color="behaviour",
    title="Analysis of Weibull shape parameter",
)
fig.show()

## Wrap up & homework <a class="anchor" id="wrap-up"></a>
Homework for next week
1. Complete the coding activities in W3-1

Your feedback today is welcome. Provide your answers in [Menti](https://www.menti.com/qusszgb46q): 
- What is one thing you liked about today?
- What would you like to see more of?
